In [ ]:
import pandas as pd

In [ ]:
import psycopg2

In [ ]:
with psycopg2.connect(user=db_user,
                  password=db_pass,
                  host="9.0.0.1",
                  port="5432",
                  connect_timeout=3,
                  database=db_name) as connection:

    hba_file = pd.read_sql_query(f'''SHOW hba_file''', connection).iloc[0,0]
    pid_file = pd.read_sql_query(f'''SHOW config_file''', connection).iloc[0,0] 

In [ ]:
hba_file, pid_file

# UNIX
```sudo service postgresql start```
# W10 (requires Admin)

In [ ]:
import win32con
import win32service
import win32serviceutil

def find_postgres_win32service():
    resume = 0
    accessSCM = win32con.GENERIC_READ
    accessSrv = win32service.SC_MANAGER_ALL_ACCESS

    #Open Service Control Manager
    hscm = win32service.OpenSCManager(None, None, accessSCM)

    #Enumerate Service Control Manager DB
    typeFilter = win32service.SERVICE_WIN32
    stateFilter = win32service.SERVICE_STATE_ALL

    statuses = win32service.EnumServicesStatus(hscm, typeFilter, stateFilter)

    for (short_name, desc, status) in statuses:
        if 'postgresql' in short_name.lower():
            return short_name, desc, status
             
#     win32serviceutil.RestartService(full_name)

In [ ]:
find_postgres_win32service()

In [3]:
import win32con
import win32service
import win32serviceutil

def restart_postgres_win32service():
    
    accessSCM = win32con.GENERIC_READ
    accessSrv = win32service.SC_MANAGER_ALL_ACCESS

    #Open Service Control Manager
    hscm = win32service.OpenSCManager(None, None, accessSCM)

    #Enumerate Service Control Manager DB
    typeFilter = win32service.SERVICE_WIN32
    stateFilter = win32service.SERVICE_STATE_ALL

    statuses = win32service.EnumServicesStatus(hscm, typeFilter, stateFilter)

    for (short_name, desc, status) in statuses:
        if 'postgresql' in short_name.lower():
            win32serviceutil.RestartService(short_name)
            return True
    return False

In [4]:
restart_postgres_win32service()

True

In [5]:
?win32serviceutil.RestartService

Signature:
win32serviceutil.RestartService(
    serviceName,
    args=None,
    waitSeconds=30,
    machine=None,
)
Docstring: Stop the service, and then start it again (with some tolerance for allowing it to stop.)
File:      c:\anaconda3\lib\site-packages\win32\lib\win32serviceutil.py
Type:      function


In [ ]:
import platform
platform.platform(terse=1)

# Try to add an entry to the existing pg_hba.conf

In [ ]:
with open(hba_file,'r') as file:
    hba = file.read()
print(hba)

In [ ]:
hba_list = hba.split('\n')
hba_list

In [ ]:
db_name = 'name'
db_user = 'user'
user_ip = '9.0.0.4'
hba_list[-1] = f"host    {db_name}    {db_user}    {user_ip}/32    md5\n"

In [ ]:
new_hba = '\n'.join(hba_list)
print(new_hba)

In [ ]:
with open("pg_hba_rmcc_backup.conf",'w') as bkup:
    bkup.write(hba)

with open(hba_file,'w') as file:
    file.write(new_hba)

In [ ]:
restart_postgres_win32service()

In [ ]:
import ipaddress
import datetime

In [ ]:
ipaddress.ip_network('9.0.0.4/32')

In [ ]:
# Default hba_conf
default_hba = (f"""# pg_hba.conf was last modified by Resolve Mission Control at {datetime.datetime.utcnow()}. Manual edit at your own risk!\n"""
                """# TYPE    DATABASE    USER    ADDRESS    METHOD\n"""
                """host    all    all    127.0.0.1/32    md5\n"""
                """host    all    all    ::1/128    md5\n""")
print(default_hba)

# Turns out [from this](https://stackoverflow.com/questions/42344890/how-to-restart-postgresql-on-os-x) a full server restart is not necessary
Just need to run select pg_reload_conf()

In [ ]:
import psycopg2

def regenerate_hba_file():
    
    with psycopg2.connect(user=db_user,
                  password=db_pass,
                  host="9.0.0.1",
                  port="5432",
                  connect_timeout=3,
                  database=db_name) as connection:

        hba_file = pd.read_sql_query(f'''SHOW hba_file''', connection).iloc[0,0]
        pid_file = pd.read_sql_query(f'''SHOW config_file''', connection).iloc[0,0] 

        # Default hba_conf
        default_hba = (f"""# pg_hba.conf was last modified by Resolve Mission Control at {datetime.datetime.utcnow()}. Manual edit at your own risk!\n"""
                        """# TYPE    DATABASE    USER    ADDRESS    METHOD\n"""
                        """host    all    all    127.0.0.1/32    md5\n"""
                        """host    all    all    ::1/128    md5\n""")

        hba_list = hba.split('\n')
        db_name = db_name
        db_user = db_user
        user_ip = '9.0.0.6'
        hba_list[-1] = f"host    {db_name}    {db_user}    {user_ip}/32    md5\n"
        new_hba = '\n'.join(hba_list)

        with open("pg_hba_rmcc_backup.conf",'w') as bkup:
            bkup.write(hba)

        with open(hba_file,'w') as file:
            file.write(new_hba)

        return pd.read_sql_query(f"select pg_reload_conf()", connection).iloc[0,0] # True if suceeded, False if failed

# macOS and without pandas

In [12]:
import psycopg2

In [16]:
with psycopg2.connect(user="postgres",
                  password="DaVinci",
                  host="127.0.0.1",
                  port="5432",
                  connect_timeout=3,
                  database='garden') as connection:
    
    crs = connection.cursor()
    crs.execute("SHOW hba_file")
    print(crs.fetchall()[0][0])
    
    crs.execute("select pg_reload_conf()")
    print(crs.fetchall()[0][0])

/Library/PostgreSQL/9.5/data/pg_hba.conf
True


In [13]:
with psycopg2.connect(user="postgres",
                      # low risk secrets
                  password="DaVinci",
                  host="127.0.0.1",
                  port="5432",
                  connect_timeout=3,
                  database='garden') as connection:
    
    crs = connection.cursor()
    crs.execute("select pg_reload_conf()")
    print(crs.fetchall()[0][0])

True


In [17]:
with open("/Library/PostgreSQL/9.5/data/pg_hba.conf", 'r') as f:
    default_hba = f.read()

In [20]:
from pathlib import Path
Path("/Library/PostgreSQL/9.5/data/pg_hba.conf").parent / Path("ahem.conf")

PosixPath('/Library/PostgreSQL/9.5/data/ahem.conf')

In [ ]:

# low risk secret
with psycopg2.connect(user="postgres",
              password="DaVinci",
              host="127.0.0.1",
              port="5432",
              connect_timeout=3,
              database=db_name) as connection:

    crs = connection.cursor()
    crs.execute("SHOW hba_file")
    hba_file = crs.fetchall()[0][0]

    # Default hba_conf
    default_hba = (f"""# pg_hba.conf was last modified by Resolve Mission Control at {datetime.datetime.utcnow()}. Manual edit at your own risk!\n"""
                    """# TYPE    DATABASE    USER    ADDRESS    METHOD\n"""
                    """host    all    all    127.0.0.1/32    md5\n""")

    hba_list = hba.split('\n')
    db_name = db_name
    db_user = db_user
    user_ip = '9.0.0.6'
    hba_list[-1] = f"host    {db_name}    {db_user}    {user_ip}/32    md5\n"
    new_hba = '\n'.join(hba_list)

    with open("pg_hba_rmcc_backup.conf",'w') as bkup:
        bkup.write(hba)

    with open(hba_file,'w') as file:
        file.write(new_hba)

    return pd.read_sql_query(f"select pg_reload_conf()", connection).iloc[0,0] # True if suceeded, False if failed

In [18]:
with open("/Library/PostgreSQL/9.5/data/pg_hba.conf", "r") as f:
    r = (f.readlines())
    r = ''.join([l for l in r if l[0] not in  ('#\n')])
    print(r)

local   all             all                                     md5
host    all             all             127.0.0.1/32            md5
host    all             all             ::1/128                 md5
host    garden    human    7.0.0.0/24    md5



In [9]:
!ping 7.0.0.4 -c 3

PING 7.0.0.4 (7.0.0.4): 56 data bytes
64 bytes from 7.0.0.4: icmp_seq=0 ttl=128 time=3.565 ms
64 bytes from 7.0.0.4: icmp_seq=1 ttl=128 time=3.327 ms
64 bytes from 7.0.0.4: icmp_seq=2 ttl=128 time=19.969 ms

--- 7.0.0.4 ping statistics ---
3 packets transmitted, 3 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 3.327/8.954/19.969/7.790 ms


pg_ctl: cannot be run as root
Please log in (using, e.g., "su") as the (unprivileged) user that will
own the server process.


In [15]:
with psycopg2.connect(user="postgres",
                  password="DaVinci",
                  host="127.0.0.1",
                  port="5432",
                  connect_timeout=3,
                  database='garden') as connection:
    
    crs = connection.cursor()
    crs.execute("select name, setting from pg_settings where name = 'data_directory';")
    print(crs.fetchall()[0][1])
    
    os.system('''su postgres -c "/Library/PostgreSQL/9.5/bin/pg_ctl restart -D /Library/PostgreSQL/9.5/data"''')

/Library/PostgreSQL/9.5/data
